In [1]:
# !pip install matplotlib tqdm numpy

In [2]:
import argparse
from pathlib import Path
import multiprocessing
from time import time
from collections import defaultdict

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tqdm import tqdm
from multiprocessor.scheduler import MultiprocessorSchedulerType

# self-written modules
from utils.metrics import MultiprocessorFeasibility, calculate_success_rate
from utils.plotters import plot_feasibility_ratio, plot_success_rate
from utils.parse import parse_task_file
from multiprocessor.feasibility.review import review_task_sets_in_parallel_multi

In [3]:
# calculate feasibility ratio by parameter
def calculate_feasibility_ratio(schedule_stats):
    feasible = schedule_stats.get(MultiprocessorFeasibility.FEASIBLE_SHORTCUT, 0) + \
               schedule_stats.get(MultiprocessorFeasibility.FEASIBLE_SIMULATION, 0)
    total = feasible + schedule_stats.get(MultiprocessorFeasibility.NOT_SCHEDULABLE_BY_A_SHORTCUT, 0) + \
            schedule_stats.get(MultiprocessorFeasibility.NOT_SCHEDULABLE_BY_A_SIMULATION, 0)
    if total == 0:
        return 0
    return feasible / total * 1.0

In [4]:
# process task sets for Plot 1 and 2 (by number of tasks)
def average_execution_time(algorithms, folder_name):
    """
    Process task sets organized by number of tasks (like in 4-tasks, 10-tasks).
    """
    subfolders = sorted([f for f in Path(folder_name).iterdir() if f.is_dir()], key=lambda x: int(x.name.split('-')[0]))
    success_rates = {alg.name: {} for alg in algorithms}

    workers = range(1, 33)

    for worker_quantity in workers:
        for subfolder in subfolders:
            print(f"Subfolder: {subfolder}")
            num_tasks = 100
            print(f"\nProcessing {num_tasks} tasks in {subfolder}")
            for alg in algorithms:
                print(f"Evaluating algorithm: {alg}")
                schedule_stats = review_task_sets_in_parallel_multi(
                    algorithm=alg, 
                    folder_name=subfolder,
                    num_processors=8, 
                    number_of_workers=worker_quantity,
                    num_clusters=4,
                    heuristic="ff"
                )
                success_rate = calculate_success_rate(schedule_stats)
                success_rates[alg.name][num_tasks] = success_rate

    return success_rates

In [ ]:
algorithms_to_evaluate = [MultiprocessorSchedulerType.PARTITIONED_EDF, MultiprocessorSchedulerType.GLOBAL_EDF, MultiprocessorSchedulerType.EDF_K]

average_execution_time(algorithms=algorithms_to_evaluate, folder_name="tasksets-multiprocessor")


Subfolder: tasksets-multiprocessor/1

Processing 100 tasks in tasksets-multiprocessor/1
Evaluating algorithm: MultiprocessorSchedulerType.GLOBAL_EDF
Checking task set: tasksets-multiprocessor/1/taskset-46
The task set tasksets-multiprocessor/1/taskset-46 is not schedulable because a necessary condition does not hold.
Checking task set: tasksets-multiprocessor/1/taskset-91


You can not tell if the task set tasksets-multiprocessor/1/taskset-91 is schedulable or not.
Checking task set: tasksets-multiprocessor/1/taskset-85
The task set tasksets-multiprocessor/1/taskset-85 is not schedulable because a necessary condition does not hold.
Checking task set: tasksets-multiprocessor/1/taskset-2
The task set tasksets-multiprocessor/1/taskset-2 is not schedulable and you had to simulate the execution.
Checking task set: tasksets-multiprocessor/1/taskset-22
The task set tasksets-multiprocessor/1/taskset-22 is not schedulable because a necessary condition does not hold.
Checking task set: tasksets-multiprocessor/1/taskset-50
